In [2]:
# https://www.kaggle.com/code/metric/santa-2023-metric
"""Evaluation metric for Santa 2023."""

import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
from sympy.combinatorics import Permutation
from typing import Dict, List


class ParticipantVisibleError(Exception):
    pass


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        series_id_column_name: str,
        moves_column_name: str,
        puzzle_info_path: str,
) -> float:
    """Santa 2023 evaluation metric.

    Parameters
    ----------
    solution : pd.DataFrame

    submission : pd.DataFrame

    series_id_column_name : str

    moves_column_name : str

    Returns
    -------
    total_num_moves : int
    """
    if list(submission.columns) != [series_id_column_name, moves_column_name]:
        raise ParticipantVisibleError(
            f"Submission must have columns {series_id_column_name} and {moves_column_name}."
        )

    puzzle_info = pd.read_csv(puzzle_info_path, index_col='puzzle_type')
    total_num_moves = 0
    for sol, sub in zip(solution.itertuples(), submission.itertuples()):
        puzzle_id = getattr(sol, series_id_column_name)
        assert puzzle_id == getattr(sub, series_id_column_name)
        allowed_moves = literal_eval(puzzle_info.loc[sol.puzzle_type, 'allowed_moves'])
        allowed_moves = {k: Permutation(v) for k, v in allowed_moves.items()}
        puzzle = Puzzle(
            puzzle_id=puzzle_id,
            allowed_moves=allowed_moves,
            solution_state=sol.solution_state.split(';'),
            initial_state=sol.initial_state.split(';'),
            num_wildcards=sol.num_wildcards,
        )

        # Score submission row
        total_num_moves += score_puzzle(puzzle_id, puzzle, getattr(sub, moves_column_name))

    return total_num_moves


@dataclass
class Puzzle:
    """A permutation puzzle."""

    puzzle_id: str
    allowed_moves: Dict[str, List[int]]
    solution_state: List[str]
    initial_state: List[str]
    num_wildcards: int


def score_puzzle(puzzle_id, puzzle, sub_solution):
    """Score the solution to a permutation puzzle."""
    # Apply submitted sequence of moves to the initial state, from left to right
    moves = sub_solution.split('.')
    state = puzzle.initial_state
    for m in moves:
        power = 1
        if m[0] == "-":
            m = m[1:]
            power = -1
        try:
            p = puzzle.allowed_moves[m]
        except KeyError:
            raise ParticipantVisibleError(f"{m} is not an allowed move for {puzzle_id}.")
        state = (p ** power)(state)

    # Check that submitted moves solve puzzle
    num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
    if num_wrong_facelets > puzzle.num_wildcards:
        raise ParticipantVisibleError(f"Submitted moves do not solve {puzzle_id}.")

    # The score for this instance is the total number of moves needed to solve the puzzle
    return len(moves)

In [3]:
import pandas as pd
from sympy.combinatorics import Permutation
from multiprocess import Pool, cpu_count

p = './santa-2023/'
path = pd.read_csv(p+'puzzles.csv')
info = pd.read_csv(p+'puzzle_info.csv')
sub = pd.read_csv(p+'sample_submission.csv')

In [4]:
path

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_33/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0


In [5]:
info

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [6]:
sub

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [7]:
info['allowed_moves_count'] = info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(path, info, how='left', on='puzzle_type')
paths = pd.merge(paths, sub, how='left', on='id')
paths['solution_state'] = paths['solution_state'].map(lambda x: x.split(';'))
paths['initial_state'] = paths['initial_state'].map(lambda x: x.split(';'))
paths['moves'] = paths['moves'].map(lambda x: x.split('.'))
paths.head(1)

,id,puzzle_type,solution_state,initial_state,num_wildcards,allowed_moves,allowed_moves_count,moves
0,0,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...",0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...","{'f0': (23)(2 19 21 8)(3 17 20 10)(4 6 7 5), '...","[r1, -f1]"


In [8]:
#Evaluation metric for Santa 2023 - https://www.kaggle.com/code/metric/santa-2023-metric
# modified - using combined paths & info dataframe

def getMoves(puzzle_id, moves, allowed_moves, state, solution_state, num_wildcards):
#     for m in moves:
#         power = 1
#         if m[0] == "-":
#             m = m[1:]
#             power = -1
#         p = allowed_moves[m]
#         state = (p ** power)(state)
#     num_wrong_facelets = sum(not(s == t) for s, t in zip(solution_state, state))
#     if num_wrong_facelets > num_wildcards:
#         print(f"Submitted moves do not solve {puzzle_id}.")
    return len(moves)

def score(sol):
    p = Pool(cpu_count()-1)
    ret = p.starmap(getMoves, sol[['id','moves','allowed_moves_count','initial_state','solution_state','num_wildcards']].values)
    p.close(); p.join()
    return sum(ret)

In [9]:
%%time
score(paths)

CPU times: total: 1.3 s
Wall time: 5.58 s


1220590

In [10]:
# Visualizing Changes
def pPositions(s):
    o = """
        Positions                              

             +--------+                               +--------+
             | .0.    .1. |                               | A    A |
             |   d1   |                               |   d1   |
             | .2.    .3. |                               | A    A |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
    | .16.    .17. | .4.    .5. | .8.    .9. | .12.    .13. |    | E    E | B    B | C    C | D    D |
    |   r1   |   f0   |   r0   |   f1   |    |   r1   |   f0   |   r0   |   f1   |
    | .18.    .19. | .6.    .7. | .10.    .11. | .14.    .15. |    | E    E | B    B | C    C | D    D |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
             | .20.    .21. |                               | F    F |
             |   d0   |                               |   d0   |
             | .22.    .23. |                               | F    F |
             +--------+                               +--------+
    """
    for i in range(len(s)):
        o = o.replace('.'+str(i)+'.', s[i])
    return o

def getMoves(args):
    puzzle_id, moves, allowed_moves, state, solution_state, num_wildcards = args
    print(f"allowed_moves: {allowed_moves}")
    for m in moves:
        print(f"move: {m}")
        power = 1
        if m[0] == "-":
            m = m[1:]
            power = -1
        p = allowed_moves[m]
        print(f"perm: {p}")
        state = (p ** power)(state)
        print(f"state: {state}")
        print(pPositions(state))
    
#     num_wrong_facelets = sum(not(s == t) for s, t in zip(solution_state, state))
#     if num_wrong_facelets > num_wildcards:
#         print(f"Submitted moves do not solve {puzzle_id}.")
    return len(moves)

getMoves(paths[['id','moves','allowed_moves_count','initial_state','solution_state','num_wildcards']].values[0])

allowed_moves: {'f0': Permutation(23)(2, 19, 21, 8)(3, 17, 20, 10)(4, 6, 7, 5), 'f1': Permutation(0, 18, 23, 9)(1, 16, 22, 11)(12, 13, 15, 14), 'r0': Permutation(1, 5, 21, 14)(3, 7, 23, 12)(8, 10, 11, 9), 'r1': Permutation(23)(0, 4, 20, 15)(2, 6, 22, 13)(16, 17, 19, 18), 'd0': Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21), 'd1': Permutation(23)(0, 1, 3, 2)(4, 16, 12, 8)(5, 17, 13, 9)}
move: r1
perm: (23)(0 4 20 15)(2 6 22 13)(16 17 19 18)
state: ['E', 'E', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'A', 'C', 'A', 'D', 'D', 'D', 'D', 'F', 'E', 'F', 'E', 'F', 'F', 'C', 'C']

        Positions                              

             +--------+                               +--------+
             | E    E |                               | A    A |
             |   d1   |                               |   d1   |
             | A    A |                               | A    A |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
    | F    E | B    B | 

2

In [11]:
# paths['moves'] = paths['moves'].map(lambda x: '.'.join(x))
# paths[['id','moves']].to_csv('submission.csv', index=False)

# 可視化

In [12]:
info.loc[0]

puzzle_type                                                   cube_2/2/2
allowed_moves          {'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...
allowed_moves_count    {'f0': (23)(2 19 21 8)(3 17 20 10)(4 6 7 5), '...
Name: 0, dtype: object

In [13]:
allowed_moves=eval(info.at[0,"allowed_moves"])
allowed_moves

{'f0': [0,
  1,
  19,
  17,
  6,
  4,
  7,
  5,
  2,
  9,
  3,
  11,
  12,
  13,
  14,
  15,
  16,
  20,
  18,
  21,
  10,
  8,
  22,
  23],
 'f1': [18,
  16,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  0,
  10,
  1,
  13,
  15,
  12,
  14,
  22,
  17,
  23,
  19,
  20,
  21,
  11,
  9],
 'r0': [0,
  5,
  2,
  7,
  4,
  21,
  6,
  23,
  10,
  8,
  11,
  9,
  3,
  13,
  1,
  15,
  16,
  17,
  18,
  19,
  20,
  14,
  22,
  12],
 'r1': [4,
  1,
  6,
  3,
  20,
  5,
  22,
  7,
  8,
  9,
  10,
  11,
  12,
  2,
  14,
  0,
  17,
  19,
  16,
  18,
  15,
  21,
  13,
  23],
 'd0': [0,
  1,
  2,
  3,
  4,
  5,
  18,
  19,
  8,
  9,
  6,
  7,
  12,
  13,
  10,
  11,
  16,
  17,
  14,
  15,
  22,
  20,
  23,
  21],
 'd1': [1,
  3,
  0,
  2,
  16,
  17,
  6,
  7,
  4,
  5,
  10,
  11,
  8,
  9,
  14,
  15,
  12,
  13,
  18,
  19,
  20,
  21,
  22,
  23]}

In [14]:
for k,v in allowed_moves.items():
    print(k, len(v))

f0 24
f1 24
r0 24
r1 24
d0 24
d1 24


In [15]:
print(allowed_moves)

{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23], 'f1': [18, 16, 2, 3, 4, 5, 6, 7, 8, 0, 10, 1, 13, 15, 12, 14, 22, 17, 23, 19, 20, 21, 11, 9], 'r0': [0, 5, 2, 7, 4, 21, 6, 23, 10, 8, 11, 9, 3, 13, 1, 15, 16, 17, 18, 19, 20, 14, 22, 12], 'r1': [4, 1, 6, 3, 20, 5, 22, 7, 8, 9, 10, 11, 12, 2, 14, 0, 17, 19, 16, 18, 15, 21, 13, 23], 'd0': [0, 1, 2, 3, 4, 5, 18, 19, 8, 9, 6, 7, 12, 13, 10, 11, 16, 17, 14, 15, 22, 20, 23, 21], 'd1': [1, 3, 0, 2, 16, 17, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 18, 19, 20, 21, 22, 23]}


In [16]:
allowed_moves.keys()

dict_keys(['f0', 'f1', 'r0', 'r1', 'd0', 'd1'])

In [17]:
path["num_wildcards"].value_counts()

0      318
2       34
4       14
6       10
8        7
10       4
16       3
18       2
42       1
34       1
38       1
54       1
176      1
12       1
Name: num_wildcards, dtype: int64

In [18]:
path.at[0,"solution_state"]

'A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F'

In [19]:
path.at[0,"initial_state"]

'D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C'

In [45]:
len(path.at[0,"initial_state"].split(";"))

24

In [44]:
len(path.at[0,"solution_state"].split(";"))

24

In [20]:
sub

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


- `allowed_moves`: 許される動き。逆動作もOK。各移動は[配列形式](https://docs.sympy.org/latest/modules/combinatorics/permutations.html)で与えられた順列。
- `*_state`: 色の配置
- `num_wildcards`: 間違えていい数
- `moves`: 動作を`.`で繋ぐ。逆動作は`-`をつける

# permutation理解

In [21]:
p = Permutation([0, 2, 1])
p

Permutation(1, 2)

In [22]:
[i^p for i in range(p.size)]

[0, 2, 1]

In [25]:
# mapping 2 to p[2]
2^p

1

In [26]:
# mapping 2 to p[2]
p(2)

1

In [27]:
# ループされる順序だけが表示される
# 要素数1は表示されない
Permutation([1, 3, 2, 0])

Permutation(0, 1, 3)

In [31]:
Permutation(0,1,3)

Permutation(0, 1, 3)

In [32]:
Permutation(1, 2)(3, 5)

Permutation(1, 2)(3, 5)

In [35]:
Permutation(1, 2)(1, 3)(2, 3)

Permutation(1, 3)

In [37]:
Permutation(1, 2)(1, 3)(2, 3).list()

[0, 3, 2, 1]

In [36]:
Permutation([0, 3, 2, 1])

Permutation(1, 3)

# 考える

In [47]:
p = './santa-2023/'
puzzles = pd.read_csv(p+'puzzles.csv')
puzzle_info = pd.read_csv(p+'puzzle_info.csv')
sample_submission = pd.read_csv(p+'sample_submission.csv')

In [51]:
df=puzzles.merge(puzzle_info, on="puzzle_type")
df

,id,puzzle_type,solution_state,initial_state,num_wildcards,allowed_moves
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
...,...,...,...,...,...,...
393,393,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,..."
394,394,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,..."
395,395,globe_33/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,..."
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0,"{'r0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,..."


In [70]:
df["initial_state"]=df["initial_state"].str.split(";")

In [58]:
df["solution_state"]=df["solution_state"].str.split(";")

In [59]:
df["state_length"]=df["solution_state"].map(len)

In [55]:
df["allowed_moves"]=df["allowed_moves"].map(eval)

In [60]:
df["allowed_action_num"]=df["allowed_moves"].map(len)

In [66]:
df["perm_length"]=df["allowed_moves"].map(lambda x: len(list(x.values())[0]))

In [68]:
(df["state_length"]==df["perm_length"]).all()

True

In [91]:
from pathlib import Path
output_dir = Path(f"./data")
output_dir.mkdir(parents=True, exist_ok=True)
for i, row in df.iterrows():
    with open(output_dir / f"{row['id']}.txt", "w") as f:
        f.write(f'{row["puzzle_type"]}\n')
        f.write(f'{row["state_length"]}\n')
        f.write(f'{" ".join(row["initial_state"])}\n')
        f.write(f'{" ".join(row["solution_state"])}\n')
        f.write(f'{row["allowed_action_num"]}\n')
        for k,v in row["allowed_moves"].items():
            f.write(f"{k} {' '.join(map(str,v))}\n")
        f.write(f'{row["num_wildcards"]}\n')